## Kindle Review Sentimental Analysis


In [2]:
import pandas as pd
import warnings
warnings.filterwarnings("ignore")


In [3]:
data = pd.read_csv('all_kindle_review.csv')

In [4]:
datacopy = data.copy()

In [5]:
datacopy.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [6]:
datacopy = datacopy[['reviewText','rating']]

In [7]:
datacopy.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [8]:
datacopy.shape

(12000, 2)

In [9]:
## cheking the null value or missing values
datacopy.isna().sum()

reviewText    0
rating        0
dtype: int64

In [10]:
## unique rating
datacopy['rating'].unique()

array([3, 5, 4, 2, 1], dtype=int64)

In [11]:
datacopy['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [12]:
## Preprocessing and cleaning
datacopy['rating'] = datacopy['rating'].apply(lambda x:0 if x<3 else 1)

In [13]:
datacopy.rating.value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [14]:
## 1. lower all the text
datacopy['reviewText'] = datacopy['reviewText'].str.lower()

In [15]:
datacopy.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [16]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ugrak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
from bs4 import BeautifulSoup

In [18]:
## removing the specil character
datacopy['reviewText'] = datacopy['reviewText'].apply(lambda x: re.sub('[^a-z A-z 0-9]+','',x))

## remove the stop words
datacopy['reviewText'] = datacopy['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))
## removing the link 
datacopy['reviewText']=datacopy['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))
## Remove html tags
datacopy['reviewText']=datacopy['reviewText'].apply(lambda x: BeautifulSoup(x, 'lxml').get_text())
## Remove any additional spaces
datacopy['reviewText']=datacopy['reviewText'].apply(lambda x: " ".join(x.split()))


In [19]:
datacopy.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


In [39]:
## Lemmatizer 
from nltk.stem import WordNetLemmatizer

In [41]:
lemmatizer = WordNetLemmatizer()

In [51]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ugrak\AppData\Roaming\nltk_data...


True

In [53]:
def lematizer_word(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

In [55]:
datacopy['reviewText'] = datacopy['reviewText'].apply(lambda x: lematizer_word(x))

In [57]:
datacopy.head()

,reviewText,rating
0,jace rankin may short he nothing mess man haul...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four book wasnt expecti...,1
3,aggie angela lansbury carry pocketbook instead...,1
4,expect type book library pleased find price right,1


In [59]:
## train test split
from sklearn.model_selection import train_test_split

In [63]:
X_train,X_test,y_train,y_test = train_test_split(datacopy['reviewText'],datacopy['rating'],test_size =0.20 ,random_state=42)

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer()

In [83]:
X_train_bow = bow.fit_transform(X_train).toarray()
X_test_bow = bow.transform(X_test).toarray()

In [85]:
## TF-IDF 
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [87]:
X_train_tfidf = tfidf.fit_transform(X_train).toarray()
X_test_tfidf = tfidf.transform(X_test).toarray()

In [89]:
X_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [101]:
# ## Naive byes algorithm 
from sklearn.naive_bayes import GaussianNB

nb_model_bow = GaussianNB().fit(X_train_bow, y_train)
# For TF-IDF
nb_model_tfidf = GaussianNB().fit(X_train_tfidf, y_train)


In [102]:
from sklearn.metrics import classification_report,accuracy_score,confusion_matrix

In [111]:
y_pred_bow = nb_model_bow.predict(X_test_bow)
y_pred_tfidf = nb_model_tfidf.predict(X_test_tfidf)

In [115]:
## print Accuracy Score
print("accuracy score for bag of words",accuracy_score(y_test,y_pred_bow))

accuracy score for bag of words 0.5779166666666666


In [117]:
confusion_matrix(y_test,y_pred_bow)

array([[500, 303],
       [710, 887]], dtype=int64)

In [119]:
## tfidf accuracy
print("accuracy score for bag of words",accuracy_score(y_test,y_pred_tfidf))

accuracy score for bag of words 0.5791666666666667


In [123]:
confusion_matrix(y_test,y_pred_tfidf)

array([[483, 320],
       [690, 907]], dtype=int64)

In [127]:
print(classification_report(y_test,y_pred_tfidf))

              precision    recall  f1-score   support

           0       0.41      0.60      0.49       803
           1       0.74      0.57      0.64      1597

    accuracy                           0.58      2400
   macro avg       0.58      0.58      0.57      2400
weighted avg       0.63      0.58      0.59      2400

